# ARI2129 Assignment - Part 1 

In [1]:
import numpy as np
import cv2

The *numpy* and *cv2* modules are imported. These two modules will be used for the entirety of this assignment.

# Stage 1

Stage 1 of this task is concerned with blending an object, taken from one image, onto a scene in another image.

Firstly, the two scenes and the object mask are loaded using *cv2* functions.

**s1** is the scene containing a single object: an ornament elephant.

**s2** is the scene containing two objects: an ornament elephant and a small ornament glass.

**mask** is the object mask for the small ornament glass.

In [2]:
s1 = cv2.imread("Images/1_colour.jpeg", 1) #Scene with 1 object
s2 = cv2.imread("Images/2_colour.jpeg") #Scene with 2 objects
s3 = cv2.imread("Images/3_colour.jpeg")
mask = cv2.imread("Images/masks/souvenirs_no_3_colour_mask_2_mask.png") #Target object mask
bg1 = cv2.imread("chairs_outdoors.jpg")
bg2 = cv2.imread("valletta_streets.jpg")
#bg3 = cv2.imread("gorg.jpg")

In [3]:
def ExtractObject(S2, ObjectMask):
    """Extracts an object from an image scene based on the mask used"""

    final_im = ObjectMask*S2
    final_im = cv2.bitwise_not(final_im)
    return final_im

In [4]:
def ApplyFilter(ExtractedObject, FilterIndex):
    """Applies convolution on the object using pre-defined image kernels"""

    if FilterIndex==0: #Apply no filter
        FilteredExObject = ExtractedObject
    elif FilterIndex==1: #Gaussian Filter
        kernel = np.ones((5, 5), np.float32)/25
        FilteredExObject = cv2.filter2D(ExtractedObject, -1, kernel)
    elif FilterIndex==2: #Histogram Equalisation Filter
        img_yuv = cv2.cvtColor(ExtractedObject, cv2.COLOR_BGR2YUV)
        img_yuv[:,:,0] = cv2.equalizeHist(img_yuv[:,:,0])
        FilteredExObject = cv2.cvtColor(img_yuv, cv2.COLOR_YUV2BGR)
    elif FilterIndex==3: #Median Blur Filter
        FilteredExObject = cv2.medianBlur(ExtractedObject, 5)
    return FilteredExObject

In [5]:
def ObjectBlender(S1, FilteredExObject):
    """Adds the filtered extracted object to the image in scene S1"""

    alpha = 0.65
    beta = 1.0 - alpha
    BlendingResult = cv2.addWeighted(S1, alpha, FilteredExObject, beta, -60)
    cv2.imshow('Blended', BlendingResult)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return BlendingResult

In [6]:
def CompareResult(BlendingResult, S2, metric):
    """Compares the blended image with the scene with 2 objects"""

    if metric==1: #Sum of Squared Distance Error (SSD)
        error = np.sum((BlendingResult-S2)**2)
    elif metric==2: #Mean Squared Error (MSE)
        error = np.sum((BlendingResult.astype("float") - S2.astype("float")) ** 2)
        error /= float(BlendingResult.shape[0] * BlendingResult.shape[1])
    return error

In [7]:
extracted = ExtractObject(s2, mask)
filtered_1 = ApplyFilter(extracted, 1)
filtered_2 = ApplyFilter(extracted, 2)
filtered_3 = ApplyFilter(extracted, 3)

The extracted object is processed using different filters. The image **s1** is blended with the 4 instances of filtered object. For each resulting image, the error scores are displayed.

In [8]:
blended_1 = ObjectBlender(s1, extracted)
blended_2 = ObjectBlender(s1, filtered_1)
blended_3 = ObjectBlender(s1, filtered_2)
blended_4 = ObjectBlender(s1, filtered_3)

In [9]:
print("No Filter")
print("SSE:", CompareResult(blended_1, s2, 1))
print("MSE:", CompareResult(blended_1, s2, 2))

print("\nGaussian Filter")
print("SSE:", CompareResult(blended_2, s2, 1))
print("MSE:", CompareResult(blended_2, s2, 2))

print("\nHistogram Equalisation Filter")
print("SSE:", CompareResult(blended_3, s2, 1))
print("MSE:", CompareResult(blended_3, s2, 2))

print("\nMedian Blur Filter")
print("SSE:", CompareResult(blended_4, s2, 1))
print("MSE:", CompareResult(blended_4, s2, 2))

No Filter
SSE: 185244013
MSE: 1328.9176182725694

Gaussian Filter
SSE: 185525091
MSE: 1333.313996310764

Histogram Equalisation Filter
SSE: 186486483
MSE: 2616.2652289496527

Median Blur Filter
SSE: 185400136
MSE: 1324.9498003472222


# Stage 2

Stage 2 of this task is concerned with removing the green background from an image and replacing it with another background.

In [22]:
def RemoveGreen(img):
    lower_bound = np.array([8, 25, 8]) #Upper bound of green colour
    upper_bound = np.array([110, 120, 60])
    cv2.normalize(img, img, 0, 255, cv2.NORM_MINMAX)
    
    temp = np.copy(img)

    mask = cv2.inRange(temp, lower_bound, upper_bound)

    masked_image = np.copy(temp)
    masked_image[mask != 0] = [0, 0, 0]

    
    return mask, masked_image

In [23]:
def NewBackground(img, NewBackground):
    """Replaces the background of an image with NewBackground"""
    
    mask, image = RemoveGreen(img)
    background_image = cv2.cvtColor(NewBackground, cv2.COLOR_BGR2RGB)

    crop_background = cv2.resize(background_image, (img.shape[1], img.shape[0]))
    crop_background[mask == 0] = [0, 0, 0]
    
    without_back = np.array(image + crop_background)
    
    cv2.imshow("Chroma keyed", without_back)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [24]:
NewBackground(s3, bg1)
NewBackground(s3, bg2)